<a href="https://colab.research.google.com/github/dhirajsuvarna/nlp_text_classification/blob/master/hackathon/Sample_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample code to predict on ULMFit Model - First Variation

Upload the Test CSV File




In [1]:
from google.colab import files
uploaded = files.upload()

Saving test_raw.csv to test_raw.csv


In [0]:
import io
import pandas as pd
import html

df_test = pd.read_csv(io.BytesIO(uploaded['test_raw.csv'])) #replace 'test.csv' with the filename.

#escaping any html codes if any
df_test['review'] = df_test['review'].apply(html.unescape)
df_test.head()

Connecting to Google Drive, since my learned model is stored there

In [3]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Load the Learned Model

In [0]:
from fastai.text import *

learn_clas = load_learner('/content/drive/My Drive/Projects/AnalyticsHackathon', 'learned_classifer_phase_1.pkl')

Iterate each of the example from Test Dataframe and perform the prediction

In [5]:
for index, row in df_test.iterrows():
  test_review = row['review']
  actual_condition = row['condition']
  pred_condition, y, pred_tensor   = learn_clas.predict(test_review)
  print (f"Review: {test_review}")
  print(f"Actual Condition: {actual_condition}")
  print(f"Predicted Codition: {pred_condition.obj}")
  print("--------------------------------------------------------------------")


Review: "I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I&#039;ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me."
Actual Condition: Depression
Predicted Codition: Depression
Review: "My son has Crohn&#039;s disease and has done very well on the Asacol.  He has no complaints and shows no side effects.  He has taken as many as nine tablets per day at one time.  I&#039;ve been very happy with the results, reducing his bouts of diarrhea drastically."
Actual Condition: Crohn's Disease, Maintenance
Predicted Codition: Crohn's Disease, Maintenance
Review: "Quick reduction of symptoms"
Actual Condition: Urinary Tract Infection
Predicted Codition: Allergic Rhinitis
R

#Sample code to Predict using ULMFit Model - Trained forward and Reverse

In [8]:
from google.colab import files
uploaded = files.upload()

Saving test_raw.csv to test_raw (1).csv


In [0]:
import io
import pandas as pd
 #replace 'test.csv' with the filename printed above after uploading
df_test = pd.read_csv(io.BytesIO(uploaded['test_raw.csv']))

Connecting to Google Drive, since my learned model is stored there

In [14]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load the Learned Model

In [0]:
from fastai.text import *

learn_clas_fwd = load_learner('/content/drive/My Drive/Projects/AnalyticsHackathon', 'learned_classifer_try_2_fwd.pkl')
learn_clas_bwd = load_learner('/content/drive/My Drive/Projects/AnalyticsHackathon', 'learned_classifer_try_2_bwd.pkl')

Iterate each of the example from Test Dataframe and perform the prediction

In [18]:
for index, row in df_test.iterrows():
  test_review = row['review']
  actual_condition = row['condition']
  pred_condition_fwd, y_fwd, pred_tensor_fwd   = learn_clas_fwd.predict(test_review)
  pred_condition_bwd, y_bwd, pred_tensor_bwd   = learn_clas_bwd.predict(test_review)

  final_pred_tensor = (pred_tensor_fwd + pred_tensor_bwd) / 2
  condition_id = np.argmax(final_pred_tensor, axis=0)
  pred_condition = learn_clas_fwd.data.classes[condition_id]

  print (f"Review: {test_review}")
  print(f"Actual Condition: {actual_condition}")
  print(f"Condition predicted by Forward Network: {pred_condition_fwd.obj}")
  print(f"Condition predicted by Backward Network: {pred_condition_bwd.obj}")
  print(f"Final Predicted Codition: {pred_condition}")
  print("--------------------------------------------------------------------")


Review: "I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I&#039;ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me."
Actual Condition: Depression
Condition predicted by Forward Network: Depression
Condition predicted by Backward Network: Depression
Final Predicted Codition: Depression
------------------------------------------------------------------------------------------------
Review: "My son has Crohn&#039;s disease and has done very well on the Asacol.  He has no complaints and shows no side effects.  He has taken as many as nine tablets per day at one time.  I&#039;ve been very happy with the results, reducing his bouts of diarrhea drastically."
Actual Co